In [1]:
'''
We need to scrape:
    For each entity
        Pokemon name
        Stats
        Moves
        item
        Status
        Condition
    For the game:
        weather
        Entry hazards
        Terrain
        What pokemon we have out
Method:
    Split text after turn into p1 update p2 update and generic update
    For each entity
        Pokemon name P1/P2
        Stats P1/P2
        Moves P1/P2
        item P1/P2
        Status P1/P2
        Condition P1/P2
    For the game:
        weather update
        Entry hazards update
        Terrain update
        What pokemon we have out update
    With request:
        Split into p1 update p2 update and update
        with p1 and p2 find "pokemon" and process string after it
        split it into the 6 identifying strings
        get
        1: Pokemon Name Identifier: ident
        2: Stats ID: "stats"
        3: Item ID: "item"
        4: Moves ID: "moves"
        5: Status ID: whitespace after health which is after condition
        From update
        get
        7: Weather 
        8: Entry hazards
        9: Terrain
        10: what pokemon we have out ID: Switch
        
        Filtering logits:
        
        Get disable moves from player request object BEFORE "pokemon"
        Get force switch from player request
        Get wait from player request: IE do nothing
    Structure of input to tensor in dict form:
        MainDict{
            "PokemonP1" : PokemonDict{
                "pkmnname": idivpkmndict{
                    "Name": pkmnname
                    "Stats" : StatDict{
                        "Health": String of x/y
                        "atk" : Int
                        "def" : Int
                        "SpA" : Int
                        "SpDef" : Int
                        "Spd": Int
                    }
                    Status: empty string if no status else status
                    Condition: same logic as status
                } 
            }
            Same with P2
            "GameEffects" : GameEffectsDict {
                Weather: String ID: -weather
                Entry Hazards: [P1 entry hazard, P2]
                pkmnout: [P1 pkmnout, P2 pkmnout]
            }
        }
        
Initial:
    |request|{"teamPreview":true,"side":{"name":"Christey","id":"p1","pokemon":[
    {"ident":"p1: Rapidash","details":"Rapidash-Galar, L84, F","condition":"246/246","active":true,"stats":{"atk":216,"def":166,"spa":183,"spd":183,"spe":225},"moves":["zenheadbutt","swordsdance","morningsun","playrough"],"baseAbility":"pastelveil","item":"leftovers","pokeball":"pokeball","ability":"pastelveil"},
    {"ident":"p1: Rotom","details":"Rotom-Fan, L84","condition":"221/221","active":false,"stats":{"atk":114,"def":228,"spa":225,"spd":228,"spe":193},"moves":["airslash","thunderbolt","willowisp","nastyplot"],"baseAbility":"levitate","item":"heavydutyboots","pokeball":"pokeball","ability":"levitate"},
    {"ident":"p1: Chandelure","details":"Chandelure, L82, M","condition":"233/233","active":false,"stats":{"atk":95,"def":195,"spa":285,"spd":195,"spe":178},"moves":["calmmind","fireblast","substitute","shadowball"],"baseAbility":"flashfire","item":"heavydutyboots","pokeball":"pokeball","ability":"flashfire"},
    {"ident":"p1: Regidrago","details":"Regidrago, L78","condition":"440/440","active":false,"stats":{"atk":201,"def":123,"spa":201,"spd":123,"spe":170},"moves":["hammerarm","dragondance","outrage","firefang"],"baseAbility":"dragonsmaw","item":"lumberry","pokeball":"pokeball","ability":"dragonsmaw"},
    {"ident":"p1: Dragonite","details":"Dragonite, L75, F","condition":"260/260","active":false,"stats":{"atk":245,"def":186,"spa":194,"spd":194,"spe":164},"moves":["earthquake","outrage","dragondance","dualwingbeat"],"baseAbility":"multiscale","item":"heavydutyboots","pokeball":"pokeball","ability":"multiscale"},
    {"ident":"p1: Luxray","details":"Luxray, L84, M","condition":"272/272","active":false,"stats":{"atk":250,"def":181,"spa":208,"spd":181,"spe":166},"moves":["facade","agility","wildcharge","crunch"],"baseAbility":"guts","item":"flameorb","pokeball":"pokeball","ability":"guts"}]}}sideupdate
Update:
    |request|
    {"active":[{"moves":[{"move":"Zen Headbutt","id":"zenheadbutt","pp":24,"maxpp":24,"target":"normal","disabled":false},{"move":"Swords Dance","id":"swordsdance","pp":32,"maxpp":32,"target":"self","disabled":false},{"move":"Morning Sun","id":"morningsun","pp":8,"maxpp":8,"target":"self","disabled":false},{"move":"Play Rough","id":"playrough","pp":16,"maxpp":16,"target":"normal","disabled":false}]}],
    "side":{"name":"Christey","id":"p1","pokemon":
    [{"ident":"p1: Rapidash","details":"Rapidash-Galar, L84, F","condition":"246/246","active":true,"stats":{"atk":216,"def":166,"spa":183,"spd":183,"spe":225},"moves":["zenheadbutt","swordsdance","morningsun","playrough"],"baseAbility":"pastelveil","item":"leftovers","pokeball":"pokeball","ability":"pastelveil"},
    {"ident":"p1: Rotom","details":"Rotom-Fan, L84","condition":"221/221","active":false,"stats":{"atk":114,"def":228,"spa":225,"spd":228,"spe":193},"moves":["airslash","thunderbolt","willowisp","nastyplot"],"baseAbility":"levitate","item":"heavydutyboots","pokeball":"pokeball","ability":"levitate"},
    {"ident":"p1: Chandelure","details":"Chandelure, L82, M","condition":"233/233","active":false,"stats":{"atk":95,"def":195,"spa":285,"spd":195,"spe":178},"moves":["calmmind","fireblast","substitute","shadowball"],"baseAbility":"flashfire","item":"heavydutyboots","pokeball":"pokeball","ability":"flashfire"},
    {"ident":"p1: Regidrago","details":"Regidrago, L78","condition":"440/440","active":false,"stats":{"atk":201,"def":123,"spa":201,"spd":123,"spe":170},"moves":["hammerarm","dragondance","outrage","firefang"],"baseAbility":"dragonsmaw","item":"lumberry","pokeball":"pokeball","ability":"dragonsmaw"},
    {"ident":"p1: Dragonite","details":"Dragonite, L75, F","condition":"260/260","active":false,"stats":{"atk":245,"def":186,"spa":194,"spd":194,"spe":164},"moves":["earthquake","outrage","dragondance","dualwingbeat"],"baseAbility":"multiscale","item":"heavydutyboots","pokeball":"pokeball","ability":"multiscale"},
    {"ident":"p1: Luxray","details":"Luxray, L84, M","condition":"272/272","active":false,"stats":{"atk":250,"def":181,"spa":208,"spd":181,"spe":166},"moves":["facade","agility","wildcharge","crunch"],"baseAbility":"guts","item":"flameorb","pokeball":"pokeball","ability":"guts"}]}}
'''

'\nWe need to scrape:\n    For each entity\n        Pokemon name\n        Stats\n        Moves\n        item\n        Status\n        Condition\n    For the game:\n        weather\n        Entry hazards\n        Terrain\n        What pokemon we have out\nMethod:\n    Split text after turn into p1 update p2 update and generic update\n    For each entity\n        Pokemon name P1/P2\n        Stats P1/P2\n        Moves P1/P2\n        item P1/P2\n        Status P1/P2\n        Condition P1/P2\n    For the game:\n        weather update\n        Entry hazards update\n        Terrain update\n        What pokemon we have out update\n    With request:\n        Split into p1 update p2 update and update\n        with p1 and p2 find "pokemon" and process string after it\n        split it into the 6 identifying strings\n        get\n        1: Pokemon Name Identifier: ident\n        2: Stats ID: "stats"\n        3: Item ID: "item"\n        4: Moves ID: "moves"\n        5: Status ID: whitespace after 

In [2]:
import tensorflow as tf
import numpy as np
%run ScrapingFunctions.ipynb

In [3]:
class NeuralNetInputFunctions:
    def ohe(val, total):
        val = int(val)
        lst = [0] * total
        lst[val] = 1
        return lst
    def oheSideEffects(conditionList):
        canonicalConditionList = []
        for condition in conditionList:
            canonicalConditionList.append(Databases.getVal(condition, Databases.sideEffects))
        lst = [0] * 5
        for num in canonicalConditionList:
            lst[num] = 1
        return lst
    def getStatList(statDict):
        statList = [statDict['atk'],
                    statDict['def'],
                    statDict['spa'],
                    statDict['spd'],
                    statDict['spe']
                   ]
        return statList
    def getMoveList(moves):
        values = []
        for i in range(4):
            try:
                moveList = Databases.getMoveList(moves[i])
                print(moveList)
                values.append(moveList[0]/999)
                values.extend(NeuralNetInputFunctions.ohe(moveList[1], 19))
                values.append(moveList[2])
            except IndexError:
                values.extend([0] * 21)
        return values
    def generatePkmnInput(playerDict):
        playerPkmnTensors = {}
        for name in list(playerDict['names'].keys()):
            values = []
            type1, type2 = Databases.getPkmnTypes(name)
            values.extend(NeuralNetInputFunctions.ohe(type1, 19))
            values.extend(NeuralNetInputFunctions.ohe(type2, 19))
            values.append(playerDict['health'][name])
            statList = NeuralNetInputFunctions.getStatList(playerDict['stats'][name])
            values.extend([i/999 for i in statList])
            moves = playerDict['moves'][name]
            values.extend(NeuralNetInputFunctions.getMoveList(moves))
            status = playerDict['status'][name]
            if status == '':
                values.extend([0] * 8)
            else:
                values.extend(NeuralNetInputFunctions.ohe(Databases.getVal(status, Databases.statuses), 8))
            playerPkmnTensors[name] = values
        return playerPkmnTensors
    def getNNheader(gameEffectDict):
        values = []
        try:
            values.extend(NeuralNetInputFunctions.ohe(Databases.getVal(gameEffectDict['weather'], Databases.weather), 7))
        except KeyError:
            values.extend([0] * 7)
        try:
            values.extend(NeuralNetInputFunctions.ohe(Databases.getVal(gameEffectDict['terrain'], Databases.terrain), 4))
        except KeyError:
            values.extend([0] * 4) 
        try:
            values.extend(NeuralNetInputFunctions.ohe(Databases.getVal(gameEffectDict['conditions'][0], Databases.conditions), 8))
        except KeyError:
            values.extend([0] * 8)
        try:
            values.extend(NeuralNetInputFunctions.ohe(Databases.getVal(gameEffectDict['conditions'][1], Databases.conditions), 8))
        except KeyError:
            values.extend([0] * 8)
        values.extend(NeuralNetInputFunctions.oheSideEffects(gameEffectDict['p1sideeffect']))
        values.extend(NeuralNetInputFunctions.oheSideEffects(gameEffectDict['p2sideeffect']))
        return values
    def getSideList(playerDict, canonicalNames):
        sideList = []
        playerPkmnDict = NeuralNetInputFunctions.generatePkmnInput(playerDict)
        for name in canonicalNames:
            sideList.extend(playerPkmnDict[name])
        return sideList
    def getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, player):
        pkmnOutP1ohe = []
        pkmnOutP2ohe = []
        nnHeader = NeuralNetInputFunctions.getNNheader(gameEffectDict)
        p1List = NeuralNetInputFunctions.getSideList(p1Dict, p1canonicalNames)
        pkmnOutP1ohe.extend(p1List)
        p2List = NeuralNetInputFunctions.getSideList(p2Dict, p2canonicalNames)
        pkmnOutP2ohe.extend(p2List)
        inpt = []
        inpt.extend(nnHeader)
        if player == 1:
            inpt.extend(pkmnOutP1ohe)
            inpt.extend(pkmnOutP2ohe)
        else:
            inpt.extend(pkmnOutP2ohe)
            inpt.extend(pkmnOutP1ohe)
        inpt = tf.convert_to_tensor(inpt, dtype=tf.float32)
        print(tf.shape(inpt))
        inpt = tf.reshape(inpt, [1, SEQUENTIALINPUTSHAPE[0]])
        return inpt

In [ ]:
class ResidaulInputFunctions:
  def getResidualInput(playerStates, numStates):
    if numStates >= 3:
      return tf.reshape(playerStates.stack()[0:3], [1, 3,1669])
    else:
      unshapedState =  tf.concat([tf.zeros([3 - numStates, INPUTSHAPE[1]], tf.float32), playerStates.stack()[0:numStates]], 0)
      return tf.reshape(unshapedState, [1, 3,1669])